# **STYLEGAN2 Implementation in 3D**

IMPORTANT NOTE: this was done using colab pro and buying an extra 300 gigabytes of space (100 which was used in training/testing over 2 weeks)



Going to start by mounting to google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Download the repository**

In [2]:
%cd /content/drive/My\ Drive/
%mkdir 3D_StyleGan
%cd 3D_StyleGan
!git clone https://github.com/KKnoernschild/StyleGan3D
%cd StyleGan3D
%cd StyleGAN_project


/content/drive/My Drive
/content/drive/My Drive/3D_StyleGan
Cloning into 'StyleGan3D'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 95 (delta 9), reused 92 (delta 9), pack-reused 0
Unpacking objects: 100% (95/95), done.
/content/drive/My Drive/3D_StyleGan/StyleGan3D
/content/drive/My Drive/3D_StyleGan/StyleGan3D/StyleGAN_project


## **Set to tensorflow 1.0**

also make sure pynrrd is installed correctly

In [3]:
#clones into project folder and installs
!git clone https://github.com/mhe/pynrrd.git
%cd pynrrd
%pip install .



Cloning into 'pynrrd'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 968 (delta 23), reused 21 (delta 10), pack-reused 924
Receiving objects: 100% (968/968), 305.04 KiB | 2.31 MiB/s, done.
Resolving deltas: 100% (573/573), done.
/content/drive/My Drive/3D_StyleGan/StyleGan3D/StyleGAN_project/pynrrd
Processing /content/drive/My Drive/3D_StyleGan/StyleGan3D/StyleGAN_project/pynrrd
  Created wheel for pynrrd: filename=pynrrd-0.4.2-py2.py3-none-any.whl size=18547 sha256=7a48a2f7c7a99af748248759d6605357f0390d75daabd6ff4567299d19e8bf3b
  Stored in directory: /tmp/pip-ephem-wheel-cache-3op4t64d/wheels/6b/f7/b4/7413c5f7a29ed6ba0d3762f37ce5dadd52d6eb5bdf7d895081
Successfully built pynrrd
  Found existing installation: pynrrd 0.4.2
    Uninstalling pynrrd-0.4.2:
      Successfully uninstalled pynrrd-0.4.2


## **Turn Your Dataset into TFRECORD Files**

call dataset_tool.py create_from_images

this takes two arguments, an output folder and then the input image folder containing the list of nrrds that you want to be converted to tfrecord format

In [0]:
#2nd argument is where to put your tfrecords files
#3rd should point at your image dataset

#EXAMPLE
!python dataset_tool.py create_from_images ./datasets/3dINHALE32 /content/drive/My\ Drive/NRRDINHALEIMS32

## **Train the system**

--num-gpus is how many gpus you have to process (we used colab so 1 in this case)

--data-dir is the file location of the dataset

--config can be a-f - we used config-f for size expansion and a large network

--dataset will be the directory you created with the tfrecord files from the cell above

--mirror-argument can be set to true of false, which just flips images along the x axis to effectively double the training image size

--metrics is set to None and does not work for our size images (something to explore in the future from sytlegan2)


In [0]:
# EXAMPLE
!python run_training.py --num-gpus=1 --data-dir=./datasets --config=config-f --dataset=3dINHALE32 --mirror-augment=true --metrics=None

you can then let this run for as long as google colab considers you not to be idle (10-19 hours in our experience)

Network Snapshots will be saved under the \results folder with a unique ID every time this line of code is ran. If a previous pkl network file is found, it will continue training on that.

## **Testing the trained newtork**

Once you feel like the network has been training for the correct amount of time, or you want to see what the results are visually to compare early epochs to later epochs, use run_generator.py generate-images

this takes 

--network : location of the pkl network file to generate images from

--seeds : seeds that you want to create images from. use commas to separate if you want individual seeds (1,2,3) or a dash to create an inclusive list of seeeds to generate images from (1-10)

--truncation-psi : this can be set anywhere from 0 to 1. 0-0.5 create 'real' images, and 0.5-1.0 tends to create more strange/maybe not quite so realisic images. 0.7 was used in our testing as a happy medium


In [0]:
# EXAMPLE
!python run_generator.py generate-images --network=/content/drive/My\ Drive/Stylegan2-colab-fresh/stylegan2/results/00039-stylegan2-3dINHALE32-1gpu-config-f/network-snapshot-024715.pkl --seeds=250-260 --truncation-psi=0.7